In [ ]:
#pip install geopandas
#pip install folium

In [ ]:
import os
import json
from urllib import request

import pandas as pd
import geopandas as gpd

import folium
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [ ]:
# импорт функции rename_subj для приведения всех названий регионов к единому стандарту
path = os.getcwd()
os.chdir("../")
from Utils.subj_rename import rename_subj
os.chdir(path.split("\\")[-1])

In [ ]:
# импорт словаря переименований регионов для унифицирования их названия в используемых df
with open("../Utils/subj_rename.json", encoding="utf-8") as file:
    subj_rename_data = json.load(file)

In [ ]:
df_divorce = pd.read_excel("divorce.xls")
df_divorce["region"] = df_divorce["region"].str.strip()

# приведение названий регионов к единому стандарту для корректного отображения на карте
df_divorce = rename_subj(
    df_divorce,
    "region",
    subj_rename_data
)
display(df_divorce.head())
df_divorce.shape

In [ ]:
df_regions = gpd.read_file("../Utils/geo_data.json").rename(columns={"name": "region"})

# приведение названий регионов к единому стандарту для корректного отображения на карте
df_regions = rename_subj(
    df_regions,
    "region",
    subj_rename_data
)
display(df_regions.head())
df_regions.shape

In [ ]:
# Соединение датафреймов для отсеивания лишних полей по столбцу region
df_show = df_divorce.merge(df_regions, on="region", how="right")
display(df_show.head())
df_show.shape

In [ ]:
df_show[df_show.k.isna()]

# Отображение на карте

In [ ]:
# [64.6863136, 97.7453061] # широта и долгота России
Map = folium.Map(
    [69, 96],
    zoom_start=3.5,
    width="%100",
    height="%100"
)

folium.Choropleth(
    geo_data=df_regions,
    data=df_show,
    name='Разводы',
    columns=['region', 'k'],   
    legend_name='Коэффициент разводимости на 1000 человек',
    key_on='feature.properties.region',
    fill_color="PuBu",
    fill_opacity=0.8,
    line_opacity=0.5,
).add_to(Map)
folium.LayerControl().add_to(Map)

tooltip = "Click Here For More Info"
# координаты середины канала - 66°25' с. ш. 94°15' в. д.
marker = folium.Marker(
    location=[66.25, 91.15],
    popup="""
        <stong>
            <a href='https://t.me/maps_stat'>tg: https://t.me/maps_stat</a><br />
            <a href='https://vk.com/maps_stat'>vk: https://vk.com/maps_stat</a>
        </stong>
    """,
    icon=folium.DivIcon(
        html="""
            <div style="color:#878787;background:#878787:90px;text-align:center;"><h5><u>@maps_stat</u><h5></div>
        """
    ),
    fill_opacity=0.5
)
marker.add_to(Map)
Map

In [ ]:
Map.save("divorce.html")

In [ ]:
# про Чукотку статья https://habr.com/ru/post/459902/

In [ ]:
import io
from PIL import Image

In [ ]:
#img_data = Map._to_png(5)
#img = Image.open(io.BytesIO(img_data))
#img.save('image.png')

In [ ]:
#img.size

In [ ]:
# crop - не пойдёт
# https://habr.com/ru/post/681248/

In [ ]:
#cr_img = img.crop(
#    (
#        250, 100,
#        1366-194, 683-50
#    )
#)

In [ ]:
#img.show()
#cr_img.show()

In [ ]:
#cr_img.save("image_cropped.png")

In [ ]:
# check later
# https://github.com/python-visualization/folium/issues/35

In [ ]:
#def _to_png(self, delay: int = 3, driver: Any = None) -> bytes:
#        """Export the HTML to byte representation of a PNG image.
#        Uses selenium to render the HTML and record a PNG. You may need to
#        adjust the `delay` time keyword argument if maps render without data or tiles.
#        Uses a headless Firefox webdriver by default, though you can provide your own.
#        Examples
#        --------
#        >>> m._to_png()
#        >>> m._to_png(time=10)  # Wait 10 seconds between render and snapshot.
#        """
#        if self._png_image is None:
#            if driver is None:
#                from selenium import webdriver
#
#                options = webdriver.firefox.options.Options()
#                options.add_argument("--headless")
#                driver = webdriver.Firefox(options=options)
#
#            html = self.get_root().render()
#            with temp_html_filepath(html) as fname:
#                # We need the tempfile to avoid JS security issues.
#                driver.get(f"file:///{fname}")
#                driver.fullscreen_window()
#                time.sleep(delay)
#                div = driver.find_element("class name", "folium-map")
#                png = div.screenshot_as_png
#                driver.quit()
#            self._png_image = png
#        return self._png_image

In [ ]:
from selenium import webdriver

from folium.utilities import (
    #TypeBounds,
    #TypeJsonValue,
    #_parse_size,
    #parse_options,
    temp_html_filepath,
    #validate_location,
)
import time

from selenium.webdriver.common.actions.pointer_actions import PointerActions
from selenium.webdriver import ActionChains

In [ ]:
def _to_png_(Map, delay: int=3, driver=None):
    """
    Export the HTML to byte representation of a PNG image.
    Uses selenium to render the HTML and record a PNG. You may need to
    adjust the `delay` time keyword argument if maps render without data or tiles.
    Uses a headless Firefox webdriver by default, though you can provide your own.
    Examples
    --------
    >>> m._to_png()
    >>> m._to_png(time=10)  # Wait 10 seconds between render and snapshot.
    """
    if driver is None:
        #from selenium import webdriver
        options = webdriver.firefox.options.Options()
        #options.add_argument("--headless")
        driver = webdriver.Firefox(options=options)
    html = Map.get_root().render()
    
    with temp_html_filepath(html) as fname:
        # We need the tempfile to avoid JS security issues.
        driver.get(f"file:///{fname}")
        driver.fullscreen_window()
        time.sleep(delay)
        div = driver.find_element("class name", "folium-map")
        png = div.screenshot_as_png
        driver.quit()
        
    return png

In [ ]:
%%time
png = _to_png_(Map, 5, driiver)

In [ ]:
img = Image.open(io.BytesIO(png))
img.save('image.png')

In [ ]:
img.show()

In [ ]:
#%%time
#Map._png_image = None
#Map._to_png(5)

In [ ]:
img = Image.open(io.BytesIO(Map._png_image))
img.save('image2.png')

In [ ]:
#img.show()

In [ ]:
%%time
options = webdriver.firefox.options.Options()
driiver = webdriver.Firefox(options=options)

In [ ]:
Map._png_image = None
Map._to_png(delay=5, driver=driiver)